In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import wikipedia
from googlesearch import search
import re
import requests
import random
import time
from itertools import chain

In [3]:
company = '03413859'

In [5]:
URL = 'https://companycheck.co.uk/company/'+company

'https://companycheck.co.uk/company/03413859'

In [6]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
referer = 'https://www.google.com'
headers = {'user-agent': user_agent, 'referer':referer}
r = requests.get(URL, timeout= 30, allow_redirects=True, headers = headers)

In [8]:
soup = BeautifulSoup(r.content, "html.parser")

In [92]:
a = soup.find_all('tr')

In [74]:
r.url

'https://companycheck.co.uk/company/03413859/AEGIS-MANAGING-AGENCY-LIMITED/companies-house-data'

In [307]:
def TAE(expr): # Trial and Error Module
    try:
        var = eval(expr)
    except:
        var = None
    return var

In [308]:
def key_financials(sub_soup, var):
    if sub_soup!=None:
        lis = sub_soup.table.find_all('tr')
        notyear = []
        year = []
        if var != 'Accounts':
            for j in lis:
                if j.find('th').text == var:
                    for i in lis[lis.index(j)].find_all('td'):
                        notyear.append(i.text)
            return notyear
        else:
            for j in lis:
                if j.find('th').text == 'Accounts':
                    for i in lis[lis.index(j)].find_all('time'):
                        year.append(i.text)
            return year
    else:
        return None

In [311]:
# Pending Task: Apply Failsafe
def CompanyCheck(ID, soup):
    if soup!=None:
        try:
            # Non returned variables
            a1 = TAE('''soup.find_all('div', class_ = 'Summary__detail')''')

            # MidSupport Variables
            if len(a1) > 0:
                Summary__detail = []
                for i in range(6):
                    Summary__detail.append((list(a1[i].stripped_strings))[1])
            else:
                Summary__detail = [None] * 6

            sub_soup = TAE('''soup.find('div', attrs = {'id': 'key-financials'})''')
            financials = {
                'accounts' : key_financials(sub_soup, 'Accounts'),
                'cash' : key_financials(sub_soup, 'Cash'),
                'net_worth' : key_financials(sub_soup, 'Net Worth'),
                'total_assets' : key_financials(sub_soup, 'Total Current Assets'),
                'total_liabilities': key_financials(sub_soup, 'Total Current Liabilities')
            }

            # Returned Variables
            company_name = Summary__detail[0]
            company_type = Summary__detail[1]
            company_status = Summary__detail[2]
            incorporated_on = Summary__detail[3]
            SIC_detail = Summary__detail[4]
            accounts = Summary__detail[5]

            accounts_year = financials['accounts'][-1]
            cash = financials['cash'][-1]
            net_worth = financials['net_worth'][-1]
            total_assets = financials['total_assets'][-1]
            total_liabilities = financials['total_liabilities'][-1]

            # EndTransform Variables

        except (IndexError, KeyError):
            data.append(None)
        data = {
            'ID':ID,
            'company_name':company_name,
            'company_type':company_type,
            'company_status':company_status,
            'incorporated_on':incorporated_on,
            'SIC_detail':SIC_detail,
            'accounts':accounts,
            'accounts_year':accounts_year,
            'cash':cash,
            'net_worth':net_worth,
            'total_assets':total_assets,
            'total_liabilities':total_liabilities
        }
    else:
        data = {
            'ID':None,
            'company_name':None,
            'company_type':None,
            'company_status':None,
            'incorporated_on':None,
            'SIC_detail':None,
            'accounts':None,
            'accounts_year':None,
            'cash':None,
            'net_worth':None,
            'total_assets':None,
            'total_liabilities':None
        }
    return [data, financials]

In [312]:
result = CompanyCheck('03413859', soup)

In [315]:
result[0]

{'ID': '03413859',
 'company_name': 'AEGIS MANAGING AGENCY LIMITED',
 'company_type': 'Private limited with Share Capital',
 'company_status': 'Active - Accounts Filed',
 'incorporated_on': '29 July 1997',
 'SIC_detail': "65120\n                Non-life insurance\n                - Lloyd's managing agency and is responsible for the management of Syndicate 1225.",
 'accounts': 'Available to 31 Dec 2018. Next accounts due by 30 Sep 2020',
 'accounts_year': '2018',
 'cash': '£95,000.00',
 'net_worth': '£23,408,000.00',
 'total_assets': '£43,852,000.00',
 'total_liabilities': '£14,879,000.00'}